# Part 1: Baseline - Understanding Batch Inference for Paper Evaluation

In this workshop, we'll build a simple paper evaluation system that:
1. **Fetches** recent AI research papers from arXiv
2. **Evaluates** each paper's relevance using the Batch API
3. **Ranks** papers to find the most interesting ones for our team

This is our **baseline** - a fixed pipeline with no decision-making.
In later parts, we'll transform this pipeline to use additional tools and do more intelligence.

---

## Setup

First, let's import our libraries and configure the environment.

**What we're using:**
- `arxiv` - Python client for searching arXiv papers
- `openai` - Client for the OpenAI-compatible Batch API. We will be using the Doubleword Batch API in our example.
- `dotenv` - Load API keys from `.env` file

In [1]:
# Standard library imports
import os
import sys
import json
import time
import re
from datetime import datetime, timedelta
from pathlib import Path

# Third-party imports
import arxiv
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Add src directory to path so we can import our modules
sys.path.insert(0, str(Path.cwd().parent / 'src'))

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


## Get our Doubleword API Key

Now we need to create a Doubleword API Key. 

1. Sign up at https://app.doubleword.com
2. Create an API key, save this in a safe place
3. Copy the `.env.example` file in the same directory, name it `.env` and add your api key 

In [2]:
# Configuration - these come from your .env file
API_KEY = os.getenv("OPENAI_API_KEY")
BASE_URL = os.getenv("OPENAI_BASE_URL")  # Optional: for non-OpenAI providers
MODEL_NAME = os.getenv("MODEL_NAME", "gpt-4o-mini")

# Verify we have an API key
if not API_KEY:
    print("⚠️  Warning: OPENAI_API_KEY not found in environment!")
    print("   Create a .env file with your API key, or we'll use sample data.")
else:
    print(f"✓ API key loaded")
    print(f"✓ Model: {MODEL_NAME}")
    if BASE_URL:
        print(f"✓ Base URL: {BASE_URL}")

✓ API key loaded
✓ Model: Qwen/Qwen3-VL-30B-A3B-Instruct-FP8
✓ Base URL: https://api.doubleword.ai/v1


Now we can run a quick test to make sure that we have established connectivity to Doubleword. The Doubleword follows the openai spec, so let's see which models we have availabile. 

In [3]:
from openai import OpenAI

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

print("Available models:")
for model in client.models.list():
    print(f"  • {model.id}")

Available models:
  • Qwen/Qwen3-VL-30B-A3B-Instruct-FP8
  • Qwen/Qwen3-Embedding-8B
  • Qwen/Qwen3-VL-235B-A22B-Instruct-FP8


### What We're Building

Our goal is to help an AI engineering team stay current with research.
Every day, hundreds of papers are published - we can't read them all!

**The Pipeline:**
```
arXiv API → Fetch Papers → Batch API → Evaluate Relevance → Ranked List to Return to User
```

**Why Batch API?**
- 50% cheaper than real-time API calls
- Process many papers in parallel
- Perfect for non-interactive workloads

---

## Step 1: Fetch Papers from arXiv

arXiv is a free repository of research papers, widely used in AI/ML.
We'll search for recent papers matching our keywords.

Let's look at the code that fetches papers:

In [ ]:
def get_papers(keywords: list[str], max_results: int = 20, days_back: int = 7) -> list[dict]:
    """
    Fetch recent papers from arXiv matching the given keywords.
    
    Args:
        keywords: Search terms (e.g., ["LLM", "transformers"])
        max_results: Maximum papers to return
        days_back: How far back to search
    
    Returns:
        List of paper dictionaries
    """
    
    # Calculate cutoff date
    cutoff_date = datetime.now() - timedelta(days=days_back)
    
    # Build search query: "keyword1" OR "keyword2" OR ...
    # Quotes ensure exact phrase matching
    query = " OR ".join([f'"{kw}"' for kw in keywords])
    print(f"🔍 Searching arXiv for: {query}")
    print(f"📅 Looking for papers since: {cutoff_date.strftime('%Y-%m-%d')}")
    
    # Configure the arXiv search
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,  # Most recent first
        sort_order=arxiv.SortOrder.Descending
    )
    
    # Execute search and collect results
    client = arxiv.Client()
    papers = []
    
    for paper in client.results(search):
        # Only include papers from our time window
        paper_date = paper.published.replace(tzinfo=None)
        if paper_date >= cutoff_date:
            papers.append({
                'id': paper.entry_id.split('/')[-1],  # e.g., "2401.12345v1"
                'title': paper.title,
                'authors': [author.name for author in paper.authors],
                'abstract': paper.summary,
                'published': paper.published.isoformat(),
                'url': paper.entry_id
            })
    
    print(f"✓ Found {len(papers)} papers")
    return papers

### Option A: Fetch Live Papers from arXiv

This searches arXiv in real-time. Results vary based on what's been published recently. You can tweak `max_results` to get more or less papers to search from

In [4]:
from get_papers import get_papers

# Define what we're interested in
SEARCH_KEYWORDS = [
    "large language models",
    "LLM agents",
    "prompt engineering"
]

# Fetch papers (set to False to use sample data instead)
USE_LIVE_DATA = True  # Change to True to fetch from arXiv

if USE_LIVE_DATA:
    papers = get_papers(SEARCH_KEYWORDS, max_results=20)
else:
    print("📂 Using sample data (set USE_LIVE_DATA = True to fetch live)")

### Option B: Load Sample Papers
For consistent results during the workshop, we'll use pre-saved sample papers.
This also works offline and doesn't require waiting for arXiv.

In [ ]:
def load_sample_papers(filepath: str = None) -> list[dict]:
    """
    Load pre-saved sample papers for offline demos.
    """
    if filepath is None:
        # Default path relative to notebooks directory
        filepath = Path.cwd().parent / "sample_data" / "papers.json"
    
    with open(filepath, 'r') as f:
        papers = json.load(f)
    
    print(f"✓ Loaded {len(papers)} sample papers")
    return papers

# Load sample papers if we didn't fetch live ones
if not USE_LIVE_DATA:
    papers = load_sample_papers()

### Explore the Papers

Let's see what we got back from arXiv (or our sample data).

In [5]:
# How many papers do we have?
print(f"Total papers: {len(papers)}")
print()

# Look at the structure of one paper
print("📄 Structure of a paper object:")
print(f"   Keys: {list(papers[0].keys())}")

Total papers: 20

📄 Structure of a paper object:
   Keys: ['id', 'title', 'authors', 'abstract', 'published', 'url']


In [6]:
# Display the first 5 papers
print("="*80)
print("FIRST 5 PAPERS")
print("="*80)

for i, paper in enumerate(papers[:5], 1):
    print(f"\n{i}. {paper['title'][:70]}..." if len(paper['title']) > 70 else f"\n{i}. {paper['title']}")
    print(f"   ID: {paper['id']}")
    print(f"   Authors: {', '.join(paper['authors'][:3])}{'...' if len(paper['authors']) > 3 else ''}")
    print(f"   Abstract: {paper['abstract'][:150]}...")

FIRST 5 PAPERS

1. Understanding and Exploiting Weight Update Sparsity for Communication-...
   ID: 2602.03839v1
   Authors: Erfan Miahi, Eugene Belilovsky
   Abstract: Reinforcement learning (RL) is a critical component for post-training large language models (LLMs). However, in bandwidth-constrained distributed RL, ...

2. Accelerating Scientific Research with Gemini: Case Studies and Common ...
   ID: 2602.03837v1
   Authors: David P. Woodruff, Vincent Cohen-Addad, Lalit Jain...
   Abstract: Recent advances in large language models (LLMs) have opened new avenues for accelerating scientific research. While models are increasingly capable of...

3. Fast-Slow Efficient Training for Multimodal Large Language Models via ...
   ID: 2602.03815v1
   Authors: Dingkun Zhang, Shuhan Qi, Yulin Wu...
   Abstract: Multimodal Large Language Models (MLLMs) suffer from severe training inefficiency issue, which is associated with their massive model sizes and visual...

4. Conformal Thinking: Risk Co

### What We Got Back

Each paper is a dictionary with:
- **id**: arXiv identifier (useful for linking back to papers)
- **title**: The paper's title
- **authors**: List of author names
- **abstract**: Summary of the paper (this is what we'll evaluate!)
- **published**: When it was submitted to arXiv
- **url**: Direct link to the paper

Now the question is: **which of these 20 papers should our team actually read?**

---

## Step 2: Batch Evaluation with AI

Now we'll use an LLM to evaluate each paper's relevance to our team.

### The Team Profile

First, we define what our team cares about. This is the "context" the AI uses to judge relevance.

In [7]:
# Define what makes a paper "relevant" for our team
# Try modifying this to see how it changes the results!

TEAM_PROFILE = {
    "focus": """
        The team is building AI-powered applications and wants to stay
        current with the latest research on language models, inference
        optimization, and practical AI engineering.
    """,
    
    "interests": [
        "Large language model architectures and improvements",
        "Inference optimization and cost reduction",
        "Prompt engineering and techniques",
        "AI agents and tool use",
        "Evaluation methods for LLMs",
    ],
    
    "avoid": [
        "Pure theoretical papers without practical applications",
        "Incremental benchmark improvements",
        "Papers focused only on training from scratch",
        "Papers focused on training models"
    ]
}

print("Team Focus:")
print(TEAM_PROFILE['focus'].strip())
print("\nInterests:")
for interest in TEAM_PROFILE['interests']:
    print(f"  • {interest}")

Team Focus:
The team is building AI-powered applications and wants to stay
        current with the latest research on language models, inference
        optimization, and practical AI engineering.

Interests:
  • Large language model architectures and improvements
  • Inference optimization and cost reduction
  • Prompt engineering and techniques
  • AI agents and tool use
  • Evaluation methods for LLMs


### The Evaluation Prompt

This is the prompt we send to the LLM for each paper.
Notice how we:
1. Give context about the team
2. Provide the paper details
3. Ask for structured JSON output

In [8]:
def create_evaluation_prompt(paper: dict, team_profile: dict) -> str:
    """
    Create the prompt that asks the AI to evaluate a paper.
    
    This is where the "magic" happens - we give the model:
    1. Context about what the team cares about
    2. The paper's title and abstract
    3. Clear instructions on how to respond
    """
    
    # Format interests and avoid lists as bullet points
    interests_text = "\n".join(f"  - {item}" for item in team_profile['interests'])
    avoid_text = "\n".join(f"  - {item}" for item in team_profile['avoid'])
    
    prompt = f"""You are evaluating research papers for an AI engineering team.

TEAM PROFILE:
{team_profile['focus'].strip()}

What they find valuable:
{interests_text}

What to avoid recommending:
{avoid_text}

---

PAPER TO EVALUATE:

Title: {paper['title']}

Abstract:
{paper['abstract']}

---

INSTRUCTIONS:
Score this paper's relevance to the team on a scale of 0-10.
- 0-3: Not relevant
- 4-6: Somewhat relevant
- 7-10: Highly relevant, team should read this

Respond with ONLY valid JSON in this exact format:
{{
    "relevance_score": <integer 0-10>,
    "key_insight": "<one sentence explaining the main takeaway>",
    "why_relevant": "<one sentence explaining why this score>"
}}"""
    
    return prompt

# Let's see what a prompt looks like
sample_prompt = create_evaluation_prompt(papers[0], TEAM_PROFILE)
print("📝 SAMPLE PROMPT (first paper):")
print("="*80)
print(sample_prompt[:1500] + "..." if len(sample_prompt) > 1500 else sample_prompt)

📝 SAMPLE PROMPT (first paper):
You are evaluating research papers for an AI engineering team.

TEAM PROFILE:
The team is building AI-powered applications and wants to stay
        current with the latest research on language models, inference
        optimization, and practical AI engineering.

What they find valuable:
  - Large language model architectures and improvements
  - Inference optimization and cost reduction
  - Prompt engineering and techniques
  - AI agents and tool use
  - Evaluation methods for LLMs

What to avoid recommending:
  - Pure theoretical papers without practical applications
  - Incremental benchmark improvements
  - Papers focused only on training from scratch
  - Papers focused on training models

---

PAPER TO EVALUATE:

Title: Understanding and Exploiting Weight Update Sparsity for Communication-Efficient Distributed RL

Abstract:
Reinforcement learning (RL) is a critical component for post-training large language models (LLMs). However, in bandwidth-const

### Creating the Batch File

The Batch API expects a JSONL file (one JSON object per line).
Each line represents one request to the API. We can either build the JSONl file using `create_batch_file` below or we can use [autobatcher](https://docs.doubleword.ai/batches/autobatcher)

#### Option A: Create Batch File

In [9]:
def create_batch_file(papers: list[dict], model: str, output_path: str = "batch_input.jsonl") -> str:
    """
    Create a JSONL file for the Batch API.
    
    Each line contains:
    - custom_id: Links response back to the paper
    - method: HTTP method (POST)
    - url: API endpoint
    - body: The actual request (model, messages, etc.)
    """
    
    with open(output_path, 'w') as f:
        for paper in papers:
            request = {
                "custom_id": paper['id'],  # We'll use this to match responses
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": model,
                    "max_tokens": 500,
                    "messages": [
                        {
                            "role": "user",
                            "content": create_evaluation_prompt(paper, TEAM_PROFILE)
                        }
                    ]
                }
            }
            f.write(json.dumps(request) + '\n')
    
    return output_path

# Create the batch file
batch_file = create_batch_file(papers, MODEL_NAME)
print(f"✓ Created batch file: {batch_file}")
print(f"  Contains {len(papers)} requests")

✓ Created batch file: batch_input.jsonl
  Contains 20 requests


In [10]:
# Let's peek at the batch file
print("📄 First 2 lines of batch file:")
print("="*80)

with open(batch_file, 'r') as f:
    for i, line in enumerate(f):
        if i >= 2:
            break
        data = json.loads(line)
        # Show a truncated version
        print(f"\nRequest {i+1}:")
        print(f"  custom_id: {data['custom_id']}")
        print(f"  method: {data['method']}")
        print(f"  model: {data['body']['model']}")
        print(f"  message length: {len(data['body']['messages'][0]['content'])} chars")

📄 First 2 lines of batch file:

Request 1:
  custom_id: 2602.03839v1
  method: POST
  model: Qwen/Qwen3-VL-30B-A3B-Instruct-FP8
  message length: 2844 chars

Request 2:
  custom_id: 2602.03837v1
  method: POST
  model: Qwen/Qwen3-VL-30B-A3B-Instruct-FP8
  message length: 2713 chars


### Submit to the Batch API

Check out the official documenation [here](https://docs.doubleword.ai/batches/getting-started-with-batched-api) on how to submit your first batch.

Now we:
1. Upload the batch file
2. Create a batch job
3. Wait for completion
4. Download results

**Note:** This requires a valid API key. If you don't have one, skip to the "Mock Results" section below.

In [11]:
def submit_batch(client: OpenAI, input_file: str) -> str:
    """
    Upload file and create a batch job.
    Returns the batch ID for tracking.
    """
    
    # Step 1: Upload the file
    print("📤 Uploading batch file...")
    with open(input_file, 'rb') as f:
        uploaded_file = client.files.create(file=f, purpose="batch")
    print(f"   File ID: {uploaded_file.id}")
    
    # Step 2: Create the batch job
    print("🚀 Creating batch job...")
    batch = client.batches.create(
        input_file_id=uploaded_file.id,
        endpoint="/v1/chat/completions",
        completion_window="1h" # this can be 24hr or 1hr 
    )
    print(f"   Batch ID: {batch.id}")
    print(f"   Status: {batch.status}")
    
    return batch.id


def wait_for_batch(client: OpenAI, batch_id: str, poll_interval: int = 10) -> str:
    """
    Poll until batch completes, then return output file ID.
    """
    
    print(f"\n⏳ Waiting for batch to complete...")
    print("   (This typically takes 1-5 minutes)\n")
    
    while True:
        batch = client.batches.retrieve(batch_id)
        status = batch.status
        
        # Show progress
        completed = (batch.request_counts.completed or 0)
        failed = (batch.request_counts.failed or 0)
        total = batch.request_counts.total or 0
        
        print(f"   Status: {status} | Progress: {completed + failed}/{total}")
        
        if status == "completed":
            print("\n✓ Batch completed!")
            return batch.output_file_id
        elif status in ["failed", "expired", "cancelled"]:
            raise Exception(f"Batch failed with status: {status}")
        
        time.sleep(poll_interval)


def download_results(client: OpenAI, output_file_id: str) -> list[dict]:
    """
    Download and parse the batch results.
    """
    
    print(f"📥 Downloading results...")
    content = client.files.content(output_file_id)
    
    results = []
    for line in content.text.strip().split('\n'):
        if line:
            results.append(json.loads(line))
    
    print(f"   Downloaded {len(results)} results")
    return results

In [12]:
# Run the batch evaluation
# Set this to True when you have a valid API key
RUN_BATCH_API = True

if RUN_BATCH_API and API_KEY:
    # Create the client
    client_kwargs = {'api_key': API_KEY}
    if BASE_URL:
        client_kwargs['base_url'] = BASE_URL
    
    client = OpenAI(**client_kwargs)
       
    # Submit and wait
    batch_id = submit_batch(client, batch_file)
    output_file_id = wait_for_batch(client, batch_id)
    raw_results = download_results(client, output_file_id)
    
    print("\n✓ Batch evaluation complete!")
else:
    print("⏭️  Skipping live API call (RUN_BATCH_API = False)")
    print("   We'll use mock results in the next cell.")


📤 Uploading batch file...
   File ID: b2a0219c-7d25-4e09-b12f-bbc8284f846a
🚀 Creating batch job...
   Batch ID: 0034c061-db71-418a-92d5-81bef945a90c
   Status: in_progress

⏳ Waiting for batch to complete...
   (This typically takes 1-5 minutes)

   Status: in_progress | Progress: 0/20
   Status: completed | Progress: 20/20

✓ Batch completed!
📥 Downloading results...
   Downloaded 20 results

✓ Batch evaluation complete!


### Option B: Use AutoBatcher 

In [15]:
# Using autobatcher for efficient batch processing
# Install: pip install autobatcher

import asyncio
from autobatcher import BatchOpenAI

async def evaluate_papers_with_autobatcher(papers, model, team_profile):
    """
    Evaluate papers using autobatcher - waits for 20 requests before sending.
    """
    
    # Create autobatcher client - batch_size=20 means it waits for 20 requests
    async with BatchOpenAI(
        api_key=API_KEY,
        base_url=BASE_URL,
        batch_size=20,              # Wait until we have 20 requests
        batch_window_seconds=60.0,  # Or send after 60 seconds if we don't hit 20
        poll_interval_seconds=5.0,  # Check for results every 5 seconds
        completion_window="1h"
    ) as client:
        
        # Create all evaluation tasks concurrently
        async def evaluate_single_paper(paper):
            prompt = create_evaluation_prompt(paper, team_profile)
            response = await client.chat.completions.create(
                model=model,
                max_tokens=500,
                messages=[{"role": "user", "content": prompt}]
            )
            content = response.choices[0].message.content
            # Parse the evaluation from the response
            try:
                evaluation = json.loads(content)
                evaluation['paper_id'] = paper['id']
                return evaluation
            except json.JSONDecodeError:
                print(f"⚠️ Failed to parse evaluation for paper {paper['id']}")
                return None
        
        # Send all 20 papers at once - autobatcher will batch them
        print(f"📤 Sending {len(papers)} papers to autobatcher (batch_size=20)...")
        tasks = [evaluate_single_paper(paper) for paper in papers]
        results = await asyncio.gather(*tasks)
        
        # Filter out any None results
        evaluations = [r for r in results if r is not None]
        print(f"✓ Received {len(evaluations)} evaluations")
        
        return evaluations
    
# Run the evaluation
print("🚀 Starting batch evaluation with autobatcher...")
evaluations = await evaluate_papers_with_autobatcher(papers, MODEL_NAME, TEAM_PROFILE)

2026-02-04 09:48:37.314 | DEBUG    | autobatcher.client:__init__:172 - Initialized with batch_size=20, window=60.0s
2026-02-04 09:48:37.316 | DEBUG    | autobatcher.client:_enqueue_request:200 - Starting 60.0s batch window timer
2026-02-04 09:48:37.317 | DEBUG    | autobatcher.client:_enqueue_request:208 - Batch size 20 reached


🚀 Starting batch evaluation with autobatcher...
📤 Sending 20 papers to autobatcher (batch_size=20)...
█████░░░░░░░░░░░░░░░  25% | Preparing batch (20 requests)...
██████████░░░░░░░░░░  50% | Uploading file...


2026-02-04 09:48:38.337 | DEBUG    | autobatcher.client:_submit_batch:277 - Uploaded batch file: 359fa9cf-4c7a-4a17-ae74-fda7c8f55032


███████████████░░░░░  75% | Creating batch...


2026-02-04 09:48:38.933 | DEBUG    | autobatcher.client:_submit_batch:287 - Submitted batch 752f656c-e03f-40ca-b5a1-85f337d6ed4b with 20 requests
2026-02-04 09:48:38.934 | DEBUG    | autobatcher.client:_poll_batches:318 - Poller started with 1 active batches


████████████████████ 100% | Submitted ✓ (batch 752f656c-e03...)


2026-02-04 09:48:44.562 | DEBUG    | autobatcher.client:_poll_batches:329 - Batch 752f656c-e03 status: completed (completed=20/20)
2026-02-04 09:48:45.686 | DEBUG    | autobatcher.client:_fetch_partial_results:423 - Resolved 20 partial results, 0 pending
2026-02-04 09:48:45.686 | DEBUG    | autobatcher.client:_poll_batches:343 - Batch 752f656c-e03f-40ca-b5a1-85f337d6ed4b completed
2026-02-04 09:48:45.687 | DEBUG    | autobatcher.client:_poll_batches:363 - Poller finished


✓ Received 20 evaluations


### Mock Results (For Workshop Demo)

If you didn't run the actual API call, here are realistic mock results
so you can see how the rest of the pipeline works.

In [ ]:
# Mock results that simulate what the API would return

MOCK_EVALUATIONS = [
    {"paper_id": "2410.21276v1", "relevance_score": 10,
     "key_insight": "Comprehensive survey of tool learning for LLM-based agents.",
     "why_relevant": "Directly addresses AI agents and tool use."},
    
    {"paper_id": "2310.04406v2", "relevance_score": 9,
     "key_insight": "AgentTuning improves agent abilities while maintaining general capabilities.",
     "why_relevant": "Practical approach to building better AI agents."},
    
    {"paper_id": "2312.10997v3", "relevance_score": 7,
     "key_insight": "Framework for governing agentic AI systems.",
     "why_relevant": "Relevant to AI agents but unclear how technical."},
    
    {"paper_id": "2401.04088v2", "relevance_score": 8,
     "key_insight": "Mixtral 8x7B achieves 6x faster inference with MoE.",
     "why_relevant": "Directly relevant to inference optimization."},
    
    {"paper_id": "2403.03507v2", "relevance_score": 9,
     "key_insight": "Claude 3 shows strong multimodal performance.",
     "why_relevant": "Important benchmark for frontier models."},
    
    {"paper_id": "2305.10601v2", "relevance_score": 9,
     "key_insight": "Tree of Thoughts enables deliberate problem solving.",
     "why_relevant": "Core technique for AI agents."},
    
    {"paper_id": "2210.03629v3", "relevance_score": 10,
     "key_insight": "ReAct synergizes reasoning and acting.",
     "why_relevant": "Foundational work on AI agents."},
    
    {"paper_id": "2305.14314v2", "relevance_score": 8,
     "key_insight": "QLoRA enables finetuning 65B models on single GPU.",
     "why_relevant": "Relevant for efficient model adaptation."},
    
    {"paper_id": "2307.09288v2", "relevance_score": 7,
     "key_insight": "Llama 2 provides open foundation models.",
     "why_relevant": "Important but abstract focuses on training."},
    
    {"paper_id": "2309.16609v2", "relevance_score": 8,
     "key_insight": "Mistral 7B outperforms larger models.",
     "why_relevant": "Relevant to inference efficiency."},
    
    {"paper_id": "2305.18290v2", "relevance_score": 6,
     "key_insight": "DPO optimizes for preferences without reward modeling.",
     "why_relevant": "Training technique - less relevant."},
    
    {"paper_id": "2402.03300v2", "relevance_score": 9,
     "key_insight": "Self-Discover lets LLMs compose reasoning structures.",
     "why_relevant": "Novel prompting for agents."},
    
    {"paper_id": "2310.06825v2", "relevance_score": 9,
     "key_insight": "Self-RAG trains models to retrieve and self-critique.",
     "why_relevant": "Practical for reducing hallucinations."},
    
    {"paper_id": "2401.02954v2", "relevance_score": 7,
     "key_insight": "DeepSeek LLM trained on 2T tokens.",
     "why_relevant": "Abstract focuses on training."},
    
    {"paper_id": "2308.12950v3", "relevance_score": 8,
     "key_insight": "Code Llama provides SOTA code generation.",
     "why_relevant": "Practical for AI-assisted coding."},
    
    {"paper_id": "2312.11805v3", "relevance_score": 8,
     "key_insight": "Gemini models trained across modalities.",
     "why_relevant": "Benchmark for multimodal development."},
    
    {"paper_id": "2310.12931v2", "relevance_score": 5,
     "key_insight": "Llemma extends Code Llama for mathematics.",
     "why_relevant": "Specialized - less relevant."},
    
    {"paper_id": "2309.05463v2", "relevance_score": 7,
     "key_insight": "phi-1.5 shows data quality trains capable small models.",
     "why_relevant": "Training insights - unclear for deployment."},
    
    {"paper_id": "2401.10020v1", "relevance_score": 9,
     "key_insight": "Survey of resource-efficient LLM techniques.",
     "why_relevant": "Directly addresses inference optimization."},
    
    {"paper_id": "2403.08295v2", "relevance_score": 7,
     "key_insight": "SIMA agents follow instructions in games.",
     "why_relevant": "Interesting but gaming-focused."}
]

# Use mock results if we didn't run the API
if not RUN_BATCH_API or not API_KEY:
    evaluations = MOCK_EVALUATIONS
    print(f"✓ Using {len(evaluations)} mock evaluations")

### Merge Results with Paper Data

Now let's combine the evaluations with the original paper data
so we have everything in one place.

In [16]:
def merge_results_with_papers(papers: list[dict], evaluations: list[dict]) -> list[dict]:
    """
    Combine paper data with evaluation results.
    Returns merged list sorted by relevance score (highest first).
    """
    
    # Create lookup for evaluations by paper_id
    eval_lookup = {e['paper_id']: e for e in evaluations}
    
    merged = []
    for paper in papers:
        paper_id = paper['id']
        if paper_id in eval_lookup:
            # Combine paper data with evaluation
            combined = {**paper, **eval_lookup[paper_id]}
            combined.pop('paper_id', None)  # Remove redundant field
            merged.append(combined)
    
    # Sort by relevance score (highest first)
    merged.sort(key=lambda x: x.get('relevance_score', 0), reverse=True)
    
    return merged

# Merge everything together
results = merge_results_with_papers(papers, evaluations)
print(f"✓ Merged {len(results)} papers with evaluations")

✓ Merged 20 papers with evaluations


### Display Top 10 Papers

Finally! Let's see which papers the AI thinks our team should read.

In [17]:
# Display the top 10 most relevant papers
TOP_N = 10
MIN_SCORE = 7  # Only show papers with score >= 7

print("="*80)
print(f"TOP {TOP_N} PAPERS FOR YOUR TEAM (score >= {MIN_SCORE})")
print("="*80)

shown = 0
for paper in results:
    score = paper.get('relevance_score', 0)
    
    if score < MIN_SCORE:
        continue
    
    shown += 1
    if shown > TOP_N:
        break
    
    # Score indicator
    if score >= 9:
        indicator = "🔥"
    elif score >= 7:
        indicator = "⭐"
    else:
        indicator = "📄"
    
    print(f"\n{indicator} [{score}/10] {paper['title'][:65]}{'...' if len(paper['title']) > 65 else ''}")
    print(f"   💡 {paper.get('key_insight', 'N/A')}")
    print(f"   📎 {paper['url']}")

print(f"\n" + "="*80)
print(f"Showing {shown} papers with relevance score >= {MIN_SCORE}")

TOP 10 PAPERS FOR YOUR TEAM (score >= 7)

⭐ [8/10] Accelerating Scientific Research with Gemini: Case Studies and Co...
   💡 The paper demonstrates practical, real-world applications of advanced LLMs in scientific research through collaborative problem-solving and novel AI-assisted methodologies.
   📎 http://arxiv.org/abs/2602.03837v1

⭐ [8/10] Conformal Thinking: Risk Control for Reasoning on a Compute Budge...
   💡 The paper introduces a risk-control framework for adaptive reasoning in LLMs that optimizes compute usage by intelligently stopping reasoning when confidence is high or problems are unsolvable, improving efficiency while maintaining accuracy.
   📎 http://arxiv.org/abs/2602.03814v1

⭐ [8/10] Antidistillation Fingerprinting
   💡 Antidistillation fingerprinting enables strong detection of model distillation with minimal impact on student model utility by aligning fingerprinting with the student's learning dynamics.
   📎 http://arxiv.org/abs/2602.03812v1

⭐ [8/10] Context Comp

---

## Step 3: Understanding the Results

Let's analyze what we got and discuss the approach.

### The JSON Structure

Each evaluated paper now contains both the original arXiv data
and the AI-generated evaluation.

In [18]:
# Look at the complete structure of a top paper
print("📊 Complete structure of a top-rated paper:")
print("="*80)

top_paper = results[0]
print(json.dumps({
    'id': top_paper['id'],
    'title': top_paper['title'][:50] + '...',
    'authors': top_paper['authors'][:2],
    'abstract': top_paper['abstract'][:100] + '...',
    'published': top_paper['published'],
    'url': top_paper['url'],
    'relevance_score': top_paper['relevance_score'],
    'key_insight': top_paper['key_insight'],
    'why_relevant': top_paper['why_relevant']
}, indent=2))

📊 Complete structure of a top-rated paper:
{
  "id": "2602.03837v1",
  "title": "Accelerating Scientific Research with Gemini: Case...",
  "authors": [
    "David P. Woodruff",
    "Vincent Cohen-Addad"
  ],
  "abstract": "Recent advances in large language models (LLMs) have opened new avenues for accelerating scientific ...",
  "published": "2026-02-03T18:56:17+00:00",
  "url": "http://arxiv.org/abs/2602.03837v1",
  "relevance_score": 8,
  "key_insight": "The paper demonstrates practical, real-world applications of advanced LLMs in scientific research through collaborative problem-solving and novel AI-assisted methodologies.",
  "why_relevant": "It highlights effective human-AI collaboration techniques, inference optimization through iterative refinement, and practical use of LLMs in complex domains, aligning with the team's interests in AI agents, prompt engineering, and inference optimization."
}


### Score Distribution

Let's see how the scores are distributed across all papers.

In [19]:
# Count papers in each score range
score_ranges = {
    'High (8-10)': 0,
    'Borderline (7)': 0,
    'Low (0-6)': 0
}

for paper in results:
    score = paper.get('relevance_score', 0)
    if score >= 8:
        score_ranges['High (8-10)'] += 1
    elif score >= 7:
        score_ranges['Borderline (7)'] += 1
    else:
        score_ranges['Low (0-6)'] += 1

print("📈 Score Distribution:")
print("="*40)
for range_name, count in score_ranges.items():
    bar = "█" * count
    print(f"{range_name:15} | {bar} ({count})")

# Average score
avg_score = sum(p.get('relevance_score', 0) for p in results) / len(results)
print(f"\nAverage score: {avg_score:.1f}")

📈 Score Distribution:
High (8-10)     | ████████ (8)
Borderline (7)  | ███████ (7)
Low (0-6)       | █████ (5)

Average score: 7.0


### The Score Distribution Problem

Notice what the average score and most papers cluster between 7-10. This is a common problem with LLM evaluation!

The model is being "polite" and hedging - everything seems "pretty good." With such low variance, it's hard to prioritize which papers to actually read.

**We need external signals to differentiate!**

---

## Part 2: Re-evaluating Borderline Papers

Papers with score **7** are borderline. We'll:
1. Download the full PDF
2. Send it to the LLM for re-evaluation
3. Get a revised score based on the full content

In [20]:
# Find borderline papers (score = 7)
import sys
sys.path.insert(0, '../src')
from tools import get_borderline_papers

borderline = get_borderline_papers(results, score=7)

print(f"Found {len(borderline)} borderline papers (score=7) to re-evaluate:")
for p in borderline:
    print(f"  • {p['title'][:55]}...")

Found 7 borderline papers (score=7) to re-evaluate:
  • Understanding and Exploiting Weight Update Sparsity for...
  • Fast-Slow Efficient Training for Multimodal Large Langu...
  • Bridging Online and Offline RL: Contextual Bandit Learn...
  • UniGeM: Unifying Data Mixing and Selection via Geometri...
  • SWE-Refactor: A Repository-Level Benchmark for Real-Wor...
  • Cognitively Diverse Multiple-Choice Question Generation...
  • Conflict-Resolving and Sharpness-Aware Minimization for...


In [21]:
from tools import get_paper_for_deep_analysis

# Step 1: Download PDFs for all borderline papers
print("📥 Downloading PDFs for borderline papers...")
papers_with_text = []

for i, paper in enumerate(borderline):
    print(f"  [{i+1}/{len(borderline)}] {paper['title'][:45]}...")
    result = get_paper_for_deep_analysis(paper, output_dir="../papers")
    papers_with_text.append(result)

success = sum(1 for p in papers_with_text if p.get('full_text'))
print(f"\n✓ Extracted text from {success}/{len(borderline)} papers")

📥 Downloading PDFs for borderline papers...
  [1/7] Understanding and Exploiting Weight Update Sp...
  ✓ Extracted 32,634 characters from 2602.03839v1
  [2/7] Fast-Slow Efficient Training for Multimodal L...
  ✓ Extracted 44,048 characters from 2602.03815v1
  [3/7] Bridging Online and Offline RL: Contextual Ba...
  ✓ Extracted 48,676 characters from 2602.03806v1
  [4/7] UniGeM: Unifying Data Mixing and Selection vi...
  ✓ Extracted 37,680 characters from 2602.03772v1
  [5/7] SWE-Refactor: A Repository-Level Benchmark fo...
  ✓ Extracted 50,336 characters from 2602.03712v1
  [6/7] Cognitively Diverse Multiple-Choice Question ...
  ✓ Extracted 30,333 characters from 2602.03704v1
  [7/7] Conflict-Resolving and Sharpness-Aware Minimi...
  ✓ Extracted 46,921 characters from 2602.03696v1

✓ Extracted text from 7/7 papers


In [28]:
from evaluate_papers import reevaluate_papers_batch

# Step 2: Batch re-evaluate all papers with the LLM
print("🤖 Submitting batch re-evaluation...")
reeval_results = reevaluate_papers_batch(papers_with_text, verbose=True)

print(f"\n✓ Got {len(reeval_results)} re-evaluation results")

🤖 Submitting batch re-evaluation...
Submitting batch of 7 papers for re-evaluation...
Batch ID: 823042a2-0b8b-4467-8d04-f4b1cbee7403
Waiting for completion...
Status: in_progress | Progress: 0/7
Status: completed | Progress: 7/7

✓ Got 7 re-evaluation results


In [29]:
# Merge re-evaluation results with original papers
reeval_lookup = {r['paper_id']: r for r in reeval_results}

print("📊 RE-EVALUATION RESULTS")
print("=" * 75)
print(f"{'Paper':<40} | {'Before':>7} | {'After':>7} | {'Change':>7}")
print("-" * 75)

final_results = []
for paper in papers_with_text:
    paper_id = paper['id']
    original_score = paper['relevance_score']
    
    if paper_id in reeval_lookup:
        revised = reeval_lookup[paper_id]
        new_score = revised.get('revised_score', original_score)
        change = new_score - original_score
        change_str = f"+{change}" if change > 0 else str(change)
        
        final_results.append({
            **paper,
            'revised_score': new_score,
            'key_insight': revised.get('key_insight'),
            'why_revised': revised.get('why_revised')
        })
    else:
        new_score = "N/A"
        change_str = "—"
        final_results.append({**paper, 'revised_score': None})
    
    title = paper['title'][:37] + "..." if len(paper['title']) > 37 else paper['title']
    print(f"{title:<40} | {original_score:>7} | {str(new_score):>7} | {change_str:>7}")

📊 RE-EVALUATION RESULTS
Paper                                    |  Before |   After |  Change
---------------------------------------------------------------------------
Understanding and Exploiting Weight U... |       7 |       8 |      +1
Fast-Slow Efficient Training for Mult... |       7 |       6 |      -1
Bridging Online and Offline RL: Conte... |       7 |       8 |      +1
UniGeM: Unifying Data Mixing and Sele... |       7 |       8 |      +1
SWE-Refactor: A Repository-Level Benc... |       7 |       7 |       0
Cognitively Diverse Multiple-Choice Q... |       7 |       8 |      +1
Conflict-Resolving and Sharpness-Awar... |       7 |       8 |      +1


In [30]:
# Final recommendations
print("\n🤖 FINAL RECOMMENDATIONS")
print("=" * 60)

# Original high-scorers
high_original = [p for p in results if p['relevance_score'] >= 8]

# Borderline papers that got upgraded
upgraded = [p for p in final_results if p.get('revised_score') and p['revised_score'] >= 8]

# Borderline papers that stayed or went down
not_upgraded = [p for p in final_results if p.get('revised_score') and p['revised_score'] < 8]

print(f"\n✅ RECOMMEND ({len(high_original) + len(upgraded)} papers):")
print("   From initial evaluation (score 8-10):")
for p in high_original[:3]:
    print(f"     • {p['title'][:50]}...")

if upgraded:
    print("   Upgraded after reading full paper:")
    for p in upgraded:
        print(f"     • {p['title'][:45]}... ({p['relevance_score']}→{p['revised_score']})")

if not_upgraded:
    print(f"\n⏭️  SKIP ({len(not_upgraded)} papers stayed borderline/downgraded):")
    for p in not_upgraded[:3]:
        print(f"     • {p['title'][:45]}... ({p['relevance_score']}→{p.get('revised_score')})")


🤖 FINAL RECOMMENDATIONS

✅ RECOMMEND (13 papers):
   From initial evaluation (score 8-10):
     • Accelerating Scientific Research with Gemini: Case...
     • Conformal Thinking: Risk Control for Reasoning on ...
     • Antidistillation Fingerprinting...
   Upgraded after reading full paper:
     • Understanding and Exploiting Weight Update Sp... (7→8)
     • Bridging Online and Offline RL: Contextual Ba... (7→8)
     • UniGeM: Unifying Data Mixing and Selection vi... (7→8)
     • Cognitively Diverse Multiple-Choice Question ... (7→8)
     • Conflict-Resolving and Sharpness-Aware Minimi... (7→8)

⏭️  SKIP (2 papers stayed borderline/downgraded):
     • Fast-Slow Efficient Training for Multimodal L... (7→6)
     • SWE-Refactor: A Repository-Level Benchmark fo... (7→7)


---

## Part 3: Gathering User Feedback

The system made recommendations, but were they actually useful? User feedback helps:
1. **Validate** the current evaluation quality
2. **Improve** future evaluations by learning what the team actually finds valuable
3. **Track** patterns in what the LLM gets wrong

This creates a feedback loop:
```
LLM Evaluation → Recommendations → User Feedback → Better Prompts/Fine-tuning
```

In [ ]:
# Interactive feedback collection using Jupyter widgets
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Store feedback
collected_feedback = []

def create_feedback_ui(papers: list[dict]):
    """Create an interactive feedback UI for rating papers."""

    # Clear previous output and feedback to avoid duplicates on re-run
    clear_output(wait=True)
    collected_feedback.clear()

    # Header
    display(HTML("""
    <h3>📝 Rate These Recommendations</h3>
    <p style="color: #666;">Click a rating for each paper. Your feedback helps improve future recommendations.</p>
    <hr>
    """))
    
    for i, paper in enumerate(papers[:5]):
        paper_id = paper['id']
        title = paper['title'][:60] + "..." if len(paper['title']) > 60 else paper['title']
        score = paper.get('relevance_score') or paper.get('revised_score', '?')
        insight = paper.get('key_insight', 'No insight available')[:80]
        
        # Paper info display
        paper_html = widgets.HTML(f"""
        <div style="background: #f5f5f5; padding: 10px; border-radius: 5px; margin: 10px 0;">
            <b style="font-size: 14px;">{i+1}. {title}</b><br>
            <span style="color: #666; font-size: 12px;">LLM Score: {score} | {insight}...</span>
        </div>
        """)
        
        # Rating buttons
        btn_not_useful = widgets.Button(description='👎 Not Useful', layout=widgets.Layout(width='120px'))
        btn_somewhat = widgets.Button(description='🤔 Okay', layout=widgets.Layout(width='100px'))
        btn_useful = widgets.Button(description='👍 Useful', layout=widgets.Layout(width='100px'))
        btn_very_useful = widgets.Button(description='⭐ Must Read', layout=widgets.Layout(width='120px'))
        
        # Status label
        status = widgets.HTML(value="<i style='color:#999'>Not rated</i>")
        
        # Button click handlers
        def make_handler(pid, ptitle, rating, rating_text, status_widget, buttons):
            def handler(b):
                collected_feedback.append({
                    'paper_id': pid,
                    'title': ptitle,
                    'user_rating': rating,
                    'rating_text': rating_text
                })
                status_widget.value = f"<span style='color:green'>✓ {rating_text}</span>"
                for btn in buttons:
                    btn.disabled = True
            return handler
        
        buttons = [btn_not_useful, btn_somewhat, btn_useful, btn_very_useful]
        btn_not_useful.on_click(make_handler(paper_id, title, 1, 'Not Useful', status, buttons))
        btn_somewhat.on_click(make_handler(paper_id, title, 2, 'Okay', status, buttons))
        btn_useful.on_click(make_handler(paper_id, title, 3, 'Useful', status, buttons))
        btn_very_useful.on_click(make_handler(paper_id, title, 4, 'Must Read', status, buttons))
        
        button_box = widgets.HBox([btn_not_useful, btn_somewhat, btn_useful, btn_very_useful, status])
        display(paper_html)
        display(button_box)
    
    display(HTML("<hr>"))
    submit_btn = widgets.Button(description='📊 Show Summary', button_style='primary')
    output = widgets.Output()
    
    def show_summary(b):
        with output:
            output.clear_output()
            if collected_feedback:
                print(f"Collected {len(collected_feedback)} ratings:")
                for f in collected_feedback:
                    print(f"  {f['title'][:40]}... → {f['rating_text']}")
            else:
                print("No ratings yet!")
    
    submit_btn.on_click(show_summary)
    display(submit_btn)
    display(output)

# Show UI for recommended papers
recommended = [p for p in results if p['relevance_score'] >= 8]
create_feedback_ui(recommended)

HTML(value='\n        <div style="background: #f5f5f5; padding: 10px; border-radius: 5px; margin: 10px 0;">\n …

HTML(value='\n        <div style="background: #f5f5f5; padding: 10px; border-radius: 5px; margin: 10px 0;">\n …

HTML(value='\n        <div style="background: #f5f5f5; padding: 10px; border-radius: 5px; margin: 10px 0;">\n …

HTML(value='\n        <div style="background: #f5f5f5; padding: 10px; border-radius: 5px; margin: 10px 0;">\n …

HTML(value='\n        <div style="background: #f5f5f5; padding: 10px; border-radius: 5px; margin: 10px 0;">\n …

Button(button_style='primary', description='📊 Show Summary', style=ButtonStyle())

Output()

In [36]:
# Analyze collected feedback
print("📊 FEEDBACK ANALYSIS")
print("=" * 50)

if collected_feedback:
    from collections import Counter
    ratings = Counter(f['user_rating'] for f in collected_feedback)
    
    print("\nRating Distribution:")
    labels = {1: '👎 Not Useful', 2: '🤔 Okay', 3: '👍 Useful', 4: '⭐ Must Read'}
    for rating, label in labels.items():
        count = ratings.get(rating, 0)
        bar = '█' * count
        print(f"  {label}: {bar} ({count})")
    
    useful = sum(1 for f in collected_feedback if f['user_rating'] >= 3)
    total = len(collected_feedback)
    print(f"\nAccuracy: {useful}/{total} ({100*useful/total:.0f}%) were useful")
else:
    print("No feedback collected yet!")

📊 FEEDBACK ANALYSIS

Rating Distribution:
  👎 Not Useful: █ (1)
  🤔 Okay:  (0)
  👍 Useful: ██ (2)
  ⭐ Must Read: ██ (2)

Accuracy: 4/5 (80%) were useful


---

## Part 4: Closing the Loop — Self-Improving Agents

This is where it gets interesting. We now have **real user feedback** on our recommendations. Instead of just hoping the LLM gets it right, we can:

1. **Detect Patterns** — Find systematic errors (e.g., "LLM keeps recommending training papers we don't care about")
2. **Update the Prompt** — Automatically suggest additions to `TEAM_PROFILE` based on what users actually value
3. **Measure Improvement** — Track accuracy over time as the prompt evolves

This is the core loop for building agents that get better:

```
┌─────────────────────────────────────────────────────────┐
│                                                         │
│   LLM Evaluates  →  User Rates  →  Find Patterns  →    │
│        ↑                                      │         │
│        └──────────  Update Prompt  ←──────────┘         │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

The agent isn't just running inference — it's **learning from feedback** to improve its own instructions.